In [1]:
import pandas as pd
import sqlite3
import numpy as np
import matplotlib.pyplot as plt


# Create your connection.
conn = sqlite3.connect('../scraping-data/database/myDB.db')
df  = pd.read_sql_query('SELECT * FROM games', conn)
# df_copy = df.copy()   


# Data Cleaning

In [2]:
# [drop-> 'Mini', 'Dungeons', 'Games', 'Warhammer', ' Word'
#Steam -> Steam Machine
# 'trading', ]

df = df.drop_duplicates(subset=['tag', 'price', 'name', 'release'])
to_drop = ['Mini', 'Dungeons', 'Games', 'Warhammer', ' Word']
for i in to_drop:
    df.drop(df[df['tag'] == i].index, inplace=True)

df['tag'] == df['tag'].replace({'Steam': 'Steam Machine'})

df1 = df.loc[
        (
        (df['tag'] == 'Trading') |   
        (df['tag'] == 'Trading Card Game')
)]

df1 = df1.drop_duplicates(subset=['name', 'type', 'release'], keep='last')

to_drop2 = ['Trading', 'Trading Card Game']
for j in to_drop2:
    df.drop(df[df['tag']== j].index, inplace=True)

df = pd.concat([df, df1])


In [3]:
#convert ['peak', 'online', 'follows'] to int type
for i in ['peak', 'online', 'follows']:
    df[i] = df[i].str.replace(',', '')
    df[i].replace({'': np.nan, '—': np.nan}, inplace=True)
    df[i] = df[i].astype('Int64')
    df[i].fillna(0, inplace=True)

df['release'].replace({'—': np.nan}, inplace=True)

#fill all missing value with Nov 199 to avoid converting problems
df['release'].fillna('Nov 1900', inplace=True)

# #while i was trying to convert this column i occured a funny release date 
# Note: delete df.drop(df[df['name'] == 'CITRON WORLD FORWARD:ZERO'].index, inplace=True) # <- this game set to release in 4444-Apr lol
df['release'] = pd.to_datetime(df['release'], format='%b %Y', errors='coerce').dt.to_period('M')
df.dropna(subset=['release'], inplace=True)


In [5]:
#try df[['rating', 'price']]
to_replace = ['rating', 'price']
for i in to_replace:
    df[i].replace({
        '—': ''
    }, inplace=True)
    
df['rating'] = pd.to_numeric(df['rating'].str.replace('%', ''))

df['price'] = pd.to_numeric(df['price'].str.replace('$', ''))
df['price'].fillna(0, inplace=True)
df['free'] = df['price'].apply(lambda x: 'free' if x == 0.00 else 'paid')

/tmp/ipykernel_4797/3796554065.py:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['price'] = pd.to_numeric(df['price'].str.replace('$', ''))


In [17]:
df.head(5)

,tag,name,type,price,rating,release,follows,online,peak,free
0,Archery,Arrow a Row,Game,0.00,94.50,2023-08,2689,347,9404,free
1,Archery,Northern Journey,Game,11.99,92.93,2021-08,3793,3,153,paid
2,Archery,The Lab,Game,0.00,91.75,2016-04,26426,18,927,free
3,Archery,TowerFall Ascension,Game,7.99,91.03,2014-03,17007,3,355,paid
4,Archery,Enderal: Forgotten Stories (Special Edition),Game,0.00,90.09,2021-03,21863,267,2428,free


In [20]:
df1 = df.drop_duplicates(subset=['name', 'type', 'release'])
df1.shape

(150733, 10)

In [38]:
too = df1.loc[df1['rating'] > 90].sort_values('rating', ascending=False)
too.shape

(1781, 10)